# Uogólnienie koncepcji MDP na przypadki ciągłe

Klasycznym przykładem problemu, który można sformułować w postaci ciągłego procesu decyzyjnego Markowa jest sterowanie odwróconego wahadła.

* Wyobraźmy sobie wózek o masie $M$, który może poruszać się po 1-wymiarowym torze i na tym wózku na zawiasie o jednym stopniu swobody umieszczony jest nieważki pręt o długości $l$ na którego końcu znajduje się punktowa masa $m$.
* Zadaniem naszym jest utrzymanie wahadła w pionie poprzez przykładanie do wózka odpowiedniej siły $F$.

![](https://brain.fuw.edu.pl/edu/images/b/b6/Cart-pendulum.png)

## Dyskretyzacja

* Przestrzeń stanu dla tego problemu jest 4-wymiarowa, bo mamy 

$$(x,\dot{x}, \theta , \dot{\theta })$$

* Najprostszym pomysłem na zastosowanie do tego przykładu (lub podobnych) metod uczenia ze wzmocnieniem (RL) jest dyskretyzacja przestrzeni stanów i przestrzeni akcji i zastosowanie algorytmów typu iteracja funkcji wartościującej lub iteracja strategii. 

* Pomysł ten napotyka w praktyce na bardzo poważny problem ze skalowaniem znany jako "klątwa wymiarów". 
Polega ona na tym, że jeśli każdy z $n$ wymiarów przestrzeni próbkujemy na $k$ poziomów to otrzymujemy siatkę o $k^{n}$ węzłów. Widać zatem, że złożoność obliczeniowa będzie rosła wykładniczo z wymiarami przestrzeni.

### Dyskretyzacja akcji

* Zazwyczaj w interesujących problemach przestrzeń akcji jest mniej wymiarowa niż przestrzeń stanów ( _pomyślmy o tym, że sterowanie obiektami w wielu grach da się zrealizować za pomocą klawiszy strzałek_ ).

### Symulator MDP
#### Model matematyczny
* Zanim przejdziemy do konkretnego algorytmu, wprowadzimy najpierw koncepcję symulatora/modelu dla MDP. 

* Model jest sposobem na opis $P_{sa}$.

* Zakładamy, że model to "czarna skrzynka" (kawałek kodu/program), która otrzymuje na wejściu stany i akcje a na wyjściu zwraca nowe stany.

$$(s,a) \rightarrow  [model] \rightarrow s'$$

W naszym przykładzie z wahadłem modelem jest układ równań opisujących dynamikę wózka i wahadła:

$$\left( M + m \right) \ddot{x} - m \ell \ddot{\theta }\cos \theta + m \ell \dot{\theta }^2 \sin \theta = F$$
$$\ell \ddot{\theta }- g \sin \theta = \ddot{x} \cos \theta $$

Możemy go przepisać jako układ równań pierwszego rzędu w którym stan jest 4-wymiarowym wektorem:

$$s = \left[
\begin{array}{ccc}
x \\
\dot{x} \\
\theta \\
\dot{\theta }\end{array}
\right]
$$

Nasz symulator mógłby działać np. całkując ten układ równań metodą Eulera pierwszego rzędu:

$$
s_{t+1} = s_{t} + \dot{s} \Delta t 
$$

#### Obserwacja modelu fizycznego
Jeśli nie umiemy wypisać równań dynamiki układu ale mamy fizyczny model i np. operatora tego modelu to można spróbować rozwiązać go przez obserwację wielu realizacji zachowania układu (model, operator), wg. poniższego algorytmu:

* Obserwujemy $m$ realizacji stanów i akcji, które doprowadziły do przejść między stanami:

$$s_0^{(1)}\, ^{\underrightarrow{a_0}^{(1)} } \, s_1^{(1)}\, ^{\underrightarrow{a_1}^{(1)} } \,  s_2^{(1)} \cdots \,  ^{\underrightarrow{a_{T-1}}^{(1)} } \, s_T^{(1)} $$
$$\vdots$$
$$s_0^{(m)}\, ^{\underrightarrow{a_0}^{(m)} } \, s_1^{(m)}\, ^{\underrightarrow{a_1}^{(m)} } \,  s_2^{(m)} \cdots \,  ^{\underrightarrow{a_{T-1}}^{(m)} } \, s_T^{(m)} $$

* Te realizacje dostarczają nam zbioru uczącego 
$\left\lbrace  (s_{t},a_{t}), s_{t+1}\right\rbrace $


*	Za pomocą jednego z algorytmów uczenia nadzorowanego (np. regresja liniowa, regresja liniowa z jądrem, sieci warstwowe z algorytmem wstecznej propagacji błędów itd.) wytwarzamy przybliżenie odwzorowania

$$s_{t+1} = f(s_{t},a_{t})$$

* _Dla ustalenia uwagi załóżmy, że w naszym przykładzie z wahadłem będzie to odwzorowanie liniowe postaci:_
$$s_{t+1} = A s_{t} + B a_{t}$$
_gdzie $A$ jest macierzą ($4 \times 4$), zaś $B$ jest wektorem (u nas 4 wymiarowym).
Zadaniem algorytmu uczącego byłoby wyznaczenie $A$ i $B$ takich, że:_

$$\arg \min _{A,B} \sum _{i=1}^{m}\sum _{t=0}^{T-1}||s_{t+1}^{(i)} - (A s_{t}^{(i)} +B a_{t}^{(i)})||^{2} $$

*	Teraz mamy dopasowany model, w wersji deterministycznej:

$$s_{t+1} = As_{t} + Ba_{t}$$

Możemy też rozważać wersję stochastyczną postaci:

$$s_{t+1} = As_{t} + Ba_{t} + \epsilon _{t}$$

gdzie $\epsilon \sim N(0,\sigma ^{2})$

## Estymacja funkcji wartościującej

* W przypadku stanów dyskretnych podawaliśmy funkcję wartościującą $V(s)$ w postaci tablicy wartości: dla każdego stanu mieliśmy przypisaną jakąś liczbę. 

* Funkcję $V$ można było znaleźć rozwiązując równania Bellmana (dla $n$ stanów mieliśmy układ $n$ równań liniowych z $n$ niewiadomymi). 
* W przypadku ciągłym nie da się zastosować tego podejścia bezpośrednio. Musimy dopasować jakąś funkcję ciągłą, która przybliżałaby $V$. 

* Można zastosować podejście analogiczne jak przy regresji:
  * Najpierw trzeba wybrać jakieś cechy $\phi (s)$, którymi będziemy charakteryzować stan $s$. 
    * W najprostszym przypadku może to być sam stan: $\phi (s) = s$ (w przypadku wahadła $[x,\dot{x}, \theta , \dot{\theta }]^{T}$). 
    * Może okazać się korzystne wzbogacenie wektora cech o jakieś dodatkowe składowe np.:
$$\phi (s) = [1, x, \dot{x}, \dot{x} ^{2}, \theta x, \dot{\theta }^{2}, \dots ]^{T}$$
* Wówczas funkcję wartościującą można przybliżyć przez:
$$V(s) = \beta ^{T} \phi (s) $$
gdzie $\beta $ to wektor parametrów regresji liniowej.

(Jest to równanie analogiczne do tego, które stosowaliśmy przy regresji liniowej $\rightarrow $ tam mieliśmy $y = h_{\theta }(x) = \theta ^{T} \phi (x)$, tu żeby nie przeciążać notacji wektor parametrów regresji nazwaliśmy $\beta $.)

Przypomnijmy, że najważniejszym punktem algorytmu iteracji funkcji wartościującej było uaktualnianie funkcji wartości zgodnie z równaniem:

$$V(s) = R(s) + \gamma \max _{a} \sum _{s^{\prime }} P_{sa}V(s^{\prime })$$

co bardziej ogólnie można wyrazić:

$$V(s) = R(s) + \gamma \max _{a} E_{s^{\prime } \sim P_{sa}}[V(s^{\prime })]$$

Widać, że przejście do przypadku ciągłych stanów wymaga znalezienia sposobu wyznaczania wartości oczekiwanej: $E_{s^{\prime } \sim P_{sa}}[V(s^{\prime })]$.

Po tym wstępie przyjrzyjmy się algorytmowi dopasowywanej funkcji wartościującej:

* Losowo próbkuj $m$ stanów $ s^{1}, s^{(2)}, \dots , s^{(m)} \in S$
* Inicjalizuj $\beta = 0$
* Powtarzaj {

$\qquad$ Dla $i = 1, \dots , m \lbrace $

$\qquad$$\qquad$Dla każdej akcji $a \in A$ {

$\qquad$$\qquad$$\qquad$Próbkuj $s_{1}^{\prime }, s_{2}^{\prime }, \dots , s_{k}^{\prime } \sim P_{s^{(i)}a}$ (używając modelu/symulatora MDP)

$\qquad$$\qquad$$\qquad$przypisz $q(a) = \frac{1}{k} \sum _{j=1}^{k} R(s^{(i)}) + \gamma V(s_{j}^{\prime })$
$\qquad$// wielkość $q(a)$ jest estymatorem $R(s) + \gamma E_{s^{\prime } \sim P_{sa}}[V(s^{\prime })]$

$\qquad$$\qquad$$\qquad$}

$\qquad$$\qquad$Przypisz $y^{(i)} = \max _{a} q(a)$$\qquad$// tak obliczone $y^{(i)}$ jest estymatorem $R(s) + \gamma \max _{a} E_{s^{\prime } \sim P_{sa}}[V(s^{\prime })]$

$\qquad$$\qquad$}

$\qquad$// w wersji dyskretnej alg. iteracji f. wartościującej uaktualnialiśmy

$\qquad$//$V(s^{(i)}) = y^{(i)}$

$\qquad$//w tym algorytmie, ponieważ $V(s)$ nie jest tablicą ale funkcją ciągłą

$\qquad$//dopasowujemy parametry f. wartościującej (jeśli jest liniowa to np. za pomocą regresji liniowej,

$\qquad$//można w tym kroku zastosować jakiś inny algorytm uczenia z nauczycielem)

$\qquad$Przypisz $\beta = \arg \min _{\beta } \frac{1}{2} \sum _{i=1}^{m}(\beta ^{T} \phi (s^{(i)}) - y(i))^{2}$

}



Zauważmy, że:
* w przypadku gdy model/symulator MDP jest deterministyczny to można uprościć powyższy algorytm kładąc $k=1$. Jest tak ponieważ wartość oczekiwana w przypadku deterministycznym jest dokładnie równa wartości obliczonej jako $V(s^{\prime })$ ($s^{\prime }$ jest jednoznaczne mając $s$ i $a$). 
* W przypadku stochastycznym trzeba empirycznie stwierdzić do jakich stanów $s^{\prime }$ przejdziemy i jaka jest średnia $V(s^{\prime })$.

Jeśli chodzi o funkcję nagrody $R(s)$ zwykle ją zadajemy zgodnie z intuicją wynikającą z problemu. W przypadku wahadła można przyjąć np.:

$$ R = \left\lbrace  \begin{array}{ r l}
-1 & \text{gdy wahadło się przewróci } \\
0 & \text{w przeciwnym razie}
\end{array} \right.$$

## Jak wyznaczyć strategię?

Algorytm estymowanej funkcji wartościującej oblicza przybliżoną postać funkcji $V^{*}$. Akcje podejmujemy wtedy w następujący sposób:

$$a = \arg \max _{a} E_{s^{\prime } \sim P_{sa} } [V(s^{\prime })]$$

W ogólnym przypadku stochastycznym dla policzenia wartości oczekiwanej w powyższym wzorze konieczne jest pobranie próbki stanów $s^{\prime } \sim P_{sa}$ i obliczenie wartości średniej $V(s^{\prime })$. Istnieją jednak dwa ważne przypadki kiedy można uprościć obliczenia wartości oczekiwanej:
* dla modelu/symulator deterministycznego do obliczenia wartości oczekiwanej wystarczy wyliczyć $V(s^{\prime })$ i wybrać tą akcję, która prowadzi do stanu $s^{\prime }$ o największej funkcji wartościującej. Zatem mamy:

$$a = \arg \max _{a} V(f(s,a))$$

* dla stochastycznego symulatora/modelu postaci:
$$s_{t+1} = f(s_{t},a_{t}) + \epsilon _{t}$$
gdzie $\epsilon _{t}$ jest próbką szumu gusowskiego $\sim N(0,\sigma ^{2})$.

W tym przypadku można zastosować przybliżenie:
$$E_{s^{\prime }\sim P_{sa}}[V^{*}(s^{\prime })] \approx V^{*}(E[s^{\prime }]) = V^{*}(f(s,a))$$

I w tym przybliżeniu akcję wybieramy jako:
$$a = \arg \max _{a} V^*(f(s,a))$$

Np. w naszym przykładzie
$$V^{*}(s) = \beta ^{T} \phi (s) $$

## Przykładowa implementacja kontrolera wahadła

Poniższy przykład zaczerpnięto z: http://mechanistician.blogspot.com/2009/05/lec17-fitted-value-iteration_31.html:


Importujemy biblioteki:

In [16]:
import matplotlib
%matplotlib --list 
%matplotlib qt
import sys
import matplotlib.pylab as plt 
import numpy as np

Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']


Najpierw napiszemy kilka funkcji pomocniczych. 

Pierwsza wytwarza wektor cech z podanego wektora stanu 

( _można tu próbować wzbogacać ten wekotr_ ):

In [17]:
def wektor_cech( x,x_dot, theta,theta_dot):
    cechy=[1.0,
            x,
            x_dot,
            theta,
            theta_dot,
            x_dot**2,
            theta_dot**2,
            theta*x]
    return np.array(cechy).reshape(1,len(cechy))

Przyda się też funkcja to testowania czy układ nie jest już za bardzo rozchwiany:

In [18]:
twelve_degrees = 0.2094384 
def is_terminal(x,theta):
    return -2.4>x or x>2.4 or -twelve_degrees>theta or theta>twelve_degrees

Teraz stworzymy klasę kontrolera. Będzie ona przydatna do przechowywania stanu i wizualizacji wahadła. Zawiera też funkcję symulującą dynamikę wahadła (model do wytwarzania przykładowych przejść między stanami). 

In [19]:
class kontroler:
    def __init__(self,stan):
        self.x = stan[0,1]
        self.x_dot = stan[0,2]
        self.theta = stan[0,3]
        self.theta_dot = stan[0,4] 
        self.proba = 0
        self.num_failures = 0
        self.dl = 3
        self.start = True
        self.g = 9.8
        self.mK = 1.0 
        self.mW = .3
        self.M=self.mW + self.mK
        self.len = .7
        self.I = self.mW *self.dl**2/3    # moment bezwl. wah.
        self.Fmag = 10.0
        self.dt = .02
        self.akcja_flip_prob = 0.0
        self.force_noise_factor = 0.0
        self.no_control_prob = 0.0
         
    def show_kontroler(self, num):
        plt.figure(1)
        
        x1 = self.x
        y1 = 0
        x2 = self.x + self.dl * np.sin(self.theta)
        y2 = self.dl * np.cos(self.theta)
           
        self.kontroler_x = [self.x-.4,self.x+.4]
        self.kontroler_y =[-.125,-.125]
        
        plt.axis([-3,3,-0.5,3.5])
        
        if self.start:
            self.wahadlo,   = plt.plot([x1,x2],[y1,y2],c='black')
            self.kontroler,= plt.plot(self.kontroler_x,self.kontroler_y,c='cyan',lw=16)
            self.start = False
        else:
            self.wahadlo.set_data([x1,x2],[y1,y2])
            self.kontroler.set_data(self.kontroler_x,self.kontroler_y)       
            plt.title('próba %d'%num)     
        plt.draw()
        plt.pause(0.01)
    
    #symulacja zachowania kontrolera
    def dynamika_kontrolera(self, akcja, stan): 
        x = stan[0,1]
        x_dot = stan[0,2]
        theta = stan[0,3]
        theta_dot = stan[0,4]
        
        if self.akcja_flip_prob > np.random.random(): #dopuszczamy błąd w kierunku siły - podmieniamy akcję na przeciwną
            akcja=1-akcja
            
        if akcja > 0:
            F = self.Fmag
        else:
            F = -self.Fmag
        
        F = F * (1 - self.force_noise_factor*np.random.uniform(-1,1)) # dopuszczamy fluktuacje siły

        if self.no_control_prob > np.random.random():
            force=0

        costheta = np.cos(theta)
        sintheta = np.sin(theta)

        temp=(F + self.I * theta_dot**2 * sintheta)/self.M
        thetaacc=(self.g*sintheta - costheta*temp)/(self.len*(4/3 -self.mW*costheta**2/self.M))
        xacc=temp - self.I*thetaacc*costheta/self.M
        
        # całkowanie ruchu Euler 1
        x += self.dt*self.x_dot
        x_dot += self.dt *xacc
        theta += self.dt *self.theta_dot
        theta_dot += self.dt *thetaacc
        return wektor_cech(x,x_dot, theta,theta_dot)
     
    def uaktualnij(self, stan):
        K.x = stan[0,1]
        K.x_dot = stan[0,2]
        K.theta = stan[0,3]
        K.theta_dot = stan[0,4]

## Symulacja 
Symulujemy zachowanie wahadła począwszy od losowego położenia początkowego wózka i losowego kąta: 

In [20]:
x = np.random.uniform(-1.1,1.1)
x_dot = 0.0
theta = np.random.uniform(-twelve_degrees,twelve_degrees)
theta_dot=0.0
stan = wektor_cech(x, x_dot, theta, theta_dot)

K = kontroler(stan)

n = np.shape(stan)[1]
m = 1000
przykladowe_stany = np.ones((m,n))

for i in range(m):
    if np.random.random()>.5:
        akcja=1
    else:
        akcja=0    
        
    stan_nowy = K.dynamika_kontrolera(akcja,stan)
    przykladowe_stany[i,:] = stan_nowy

    if is_terminal(stan_nowy[0,1],stan_nowy[0,3]):
        x = np.random.uniform(-1.1,1.1)
        x_dot = 0.0
        theta = np.random.uniform(-twelve_degrees,twelve_degrees)
        theta_dot = 0.0
        stan = wektor_cech(x, x_dot, theta, theta_dot)
        K.uaktualnij(stan)
    else:
        stan = stan_nowy
        K.uaktualnij(stan) 
        K.show_kontroler(i)  

## Estymacja funkcji wartościującej
Iteracyjnie estymujemy funkcję $V^*$. $$V^{*}(s) = \beta ^{T} \phi (s) $$ 
W każdym kroku iteracji za pomocą metody najmniejszych kwadratów znajdujemy najlepsze parametry $\beta$:

In [21]:
gamma = 0.995
diff = 10**10   
new_diff = diff-1
epochs = 0
beta = np.zeros((n,1))

while 10 > epochs and diff > new_diff:
    diff = new_diff
    epochs += 1
    print(epochs)

    y_vec = np.zeros((m,1))   
    
    for i in range(m):
        stan = przykladowe_stany[i,:].reshape(1,n)

        # POBRANIE NAGRODY DLA AKTUALNEGO STANU
        if is_terminal(stan[0,1],stan[0,3]):
            R=-1.0
        else:
            R=0.0         
        
        # OBLICZENIE FUNKCJI WARTOŚCIUJĄCEJ DLA AKCJI = 0 
        nowy_stan0 = K.dynamika_kontrolera(0, stan)
        V1 = R + gamma*np.dot(nowy_stan0, beta)
        
        # OBLICZENIE FUNKCJI WARTOŚCIUJĄCEJ DLA AKCJI = 1 
        nowy_stan1 = K.dynamika_kontrolera(1, stan)
        V2 = R + gamma*np.dot(nowy_stan1, beta)
        
        # \arg \max _{a} V(f(s,a))
        if V1 > V2:
            akcja=0
            V = V1
        elif V2 > V1:
            akcja = 1
            V = V2
        else:
            if np.random.random()>.5:
                akcja = 1
                V = V2
            else:
                akcja=0
                V = V1
        y_vec[i,0] = V
    
    result = np.linalg.lstsq(przykladowe_stany,y_vec) # tu fitujemy model liniowy
    beta = result[0]
    new_diff = result[1]

1
2


/var/folders/1t/qbjskh415rz70kx6_0rww8680000gp/T/ipykernel_68601/3658986060.py:47: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  result = np.linalg.lstsq(przykladowe_stany,y_vec) # tu fitujemy model liniowy


## Ilustracja działania nauczonego kontrolera

In [22]:
plt.ion()
t = 0

stan = wektor_cech(x = 0, x_dot = 0, theta= 0, theta_dot = 0)
K.uaktualnij(stan)

while True:    
    stan0 = K.dynamika_kontrolera(0,stan)
    q1 = np.dot(stan0, beta)
    
    stan1= K.dynamika_kontrolera(1, stan)
    q2 = np.dot(stan1, beta)
    
    if q1 > q2:
        akcja = 0
    elif q2 > q1:
        akcja = 1
    else:
        if np.random.random()>.5:
            akcja=1
        else:
            akcja=0

    stan = K.dynamika_kontrolera(akcja,stan)
    K.uaktualnij(stan)
    K.show_kontroler(t)
    t+=1  
    if is_terminal(stan[0,1],stan[0,3]):
        break
plt.ioff() 

qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7f9a728e8b70> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x7f9a728e8b70> contents scale of 2 - updating layer to match.
